In [2]:
# Transfer Learning con MobileNetV2

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Deep Learning
import tensorflow as tf
import tensorflow
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing import image_dataset_from_directory

# dataset
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
data_dir = tf.keras.utils.get_file("cats_and_dogs_filtered.zip", origin = dataset_url,
                                   extract = True,
                                   cache_subdir = os.getcwd())


68606236/68606236 [==============================] - 1s 0us/step


In [3]:
# Obtener la ruta del directorio donde se descomprimio la data
data_dir = os.path.join(os.path.dirname(data_dir), "cats_and_dogs_filtered")
data_dir

'/content/cats_and_dogs_filtered'

In [4]:
# Definamos un tamaño estandar para la imagenes y un batch_size
img_size = (160,160)
batch_size = 32

In [5]:
# Carguemos los datos para entrenar y validar nuestra red neuronal
train_dataset = image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "training",
    seed = 666,
    image_size = img_size,
    batch_size = batch_size
)

validation_dataset = image_dataset_from_directory(
    data_dir,
    validation_split = 0.2,
    subset = "validation",
    seed = 666,
    image_size = img_size,
    batch_size = batch_size
)

Found 3000 files belonging to 2 classes.
Using 2400 files for training.
Found 3000 files belonging to 2 classes.
Using 600 files for validation.


In [6]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:
# Configuremos el AUTOTUNE
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size = AUTOTUNE)

In [8]:
# Carguemos en memoria el modelo MovileNetV2
pretrained_model = tf.keras.applications.MobileNetV2(input_shape = (160,160, 3),
                                                     include_top = False,
                                                     weights = "imagenet")

type(pretrained_model)

9406464/9406464 [==============================] - 0s 0us/step


keras.src.engine.functional.Functional

In [9]:
# Vamos a crear un modelo usando la clase Sequential
model = models.Sequential([
    pretrained_model,
    # MOdifiquemos el clasificador
    layers.GlobalAveragePooling2D(),
    layers.Dense(1)
])

In [11]:
#
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_160 (Func  (None, 5, 5, 1280)        2257984   
 tional)                                                         
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2259265 (8.62 MB)
Trainable params: 2225153 (8.49 MB)
Non-trainable params: 34112 (133.25 KB)
_________________________________________________________________


In [16]:
# Compilemos nuestra arquitectura (Version modificada de mobileNetV2)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(from_logits = True),
              optimizer = optimizers.Adam(learning_rate = 10**(-5)),
              metrics = ["accuracy"])

# Entrenamiento del modelo
HistoriaAjuste = model.fit(train_dataset,
                           epochs = 10,
                           validation_data = validation_dataset)

Epoch 1/10
75/75 [==============================] - 248s 3s/step - loss: 0.7665 - accuracy: 0.5725 - val_loss: 1.0590 - val_accuracy: 0.4433
Epoch 2/10
75/75 [==============================] - 222s 3s/step - loss: 0.6532 - accuracy: 0.6646 - val_loss: 0.9276 - val_accuracy: 0.4983
Epoch 3/10
75/75 [==============================] - 228s 3s/step - loss: 0.6016 - accuracy: 0.6829 - val_loss: 0.8578 - val_accuracy: 0.5400
Epoch 4/10
75/75 [==============================] - 222s 3s/step - loss: 0.5628 - accuracy: 0.6929 - val_loss: 0.8292 - val_accuracy: 0.5417
Epoch 5/10
75/75 [==============================] - 221s 3s/step - loss: 0.5320 - accuracy: 0.7008 - val_loss: 0.8189 - val_accuracy: 0.5317
Epoch 6/10
75/75 [==============================] - 221s 3s/step - loss: 0.4967 - accuracy: 0.7188 - val_loss: 0.8015 - val_accuracy: 0.5417
Epoch 7/10
75/75 [==============================] - 227s 3s/step - loss: 0.4673 - accuracy: 0.7300 - val_loss: 0.7959 - val_accuracy: 0.5683
Epoch 8/10
75